<a href="https://colab.research.google.com/github/z-zaremska/PK_ML_Bootcamp/blob/master/supervised%5C01_basics%5C01_preprocessing_zz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import pandas as pd
import sklearn

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, scale, StandardScaler

sklearn.__version__

'1.6.1'

In [2]:
data = {
    'size': ['XL', 'L', 'M', 'L', 'M'],
    'color': ['red', 'green', 'blue', 'green', 'red'],
    'gender': ['female', 'male', 'male', 'female', 'female'],
    'price': [199.0, 89.0, 99.0, 129.0, 79.0],
    'weight': [500, 450, 300, 380, 410],
    'bought': ['yes', 'no', 'yes', 'no', 'yes']
}

df_raw = pd.DataFrame(data=data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [3]:
df = df_raw.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 372.0+ bytes


In [4]:
cols = ["size", "color", "gender", "bought"]
for column in cols:
  df[column] = df[column].astype("category")

df["weight"] = df["weight"].astype("float")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   size    5 non-null      category
 1   color   5 non-null      category
 2   gender  5 non-null      category
 3   price   5 non-null      float64 
 4   weight  5 non-null      float64 
 5   bought  5 non-null      category
dtypes: category(4), float64(2)
memory usage: 744.0 bytes


In [5]:
df.describe()

,price,weight
count,5.000000,5.000000
mean,119.000000,408.000000
std,48.476799,75.299402
min,79.000000,300.000000
25%,89.000000,380.000000
50%,99.000000,410.000000
75%,129.000000,450.000000
max,199.000000,500.000000


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,5.0,119.0,48.476799,79.0,89.0,99.0,129.0,199.0
weight,5.0,408.0,75.299402,300.0,380.0,410.0,450.0,500.0


In [7]:
df.describe(include=["category"]).T

,count,unique,top,freq
size,5,3,L,2
color,5,3,green,2
gender,5,2,female,3
bought,5,2,yes,3


In [13]:
le = LabelEncoder()
# le.fit(df['bought'])
# le.transform(df['bought'])
le.fit_transform(df['bought'])

array([1, 0, 1, 0, 1])

In [14]:
le.classes_

array(['no', 'yes'], dtype=object)

In [15]:
df["bought"] = le.fit_transform(df["bought"])
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


In [24]:
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(df[["size"]])
encoder.transform(df[["size"]])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [25]:
encoder.categories_

[array(['L', 'M', 'XL'], dtype=object)]

In [26]:
encoder = OneHotEncoder(sparse_output=False, drop="first") # When encoding the category column the number of array column is n-1
encoder.fit(df[["size"]])
encoder.transform(df[["size"]])

array([[0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.]])

In [29]:
encoder.categories_

[array(['L', 'M', 'XL'], dtype=object)]

In [31]:
df["bought"] = le.inverse_transform(df["bought"])
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,yes
1,L,green,male,89.0,450.0,no
2,M,blue,male,99.0,300.0,yes
3,L,green,female,129.0,380.0,no
4,M,red,female,79.0,410.0,yes


In [32]:
pd.get_dummies(data=df)

,price,weight,size_L,size_M,size_XL,color_blue,color_green,color_red,gender_female,gender_male,bought_no,bought_yes
0,199.0,500.0,False,False,True,False,False,True,True,False,False,True
1,89.0,450.0,True,False,False,False,True,False,False,True,True,False
2,99.0,300.0,False,True,False,True,False,False,False,True,False,True
3,129.0,380.0,True,False,False,False,True,False,True,False,True,False
4,79.0,410.0,False,True,False,False,False,True,True,False,False,True


In [35]:
pd.get_dummies(data=df, drop_first=True)

,price,weight,size_M,size_XL,color_green,color_red,gender_male,bought_yes
0,199.0,500.0,False,True,False,True,False,True
1,89.0,450.0,False,False,True,False,True,False
2,99.0,300.0,True,False,False,False,True,True
3,129.0,380.0,False,False,True,False,False,False
4,79.0,410.0,True,False,False,True,False,True


In [37]:
pd.get_dummies(data=df, drop_first=True, columns=["bought"])

,size,color,gender,price,weight,bought_yes
0,XL,red,female,199.0,500.0,True
1,L,green,male,89.0,450.0,False
2,M,blue,male,99.0,300.0,True
3,L,green,female,129.0,380.0,False
4,M,red,female,79.0,410.0,True


### Standaryzacja

In [43]:
print(f"{df['price']}\n")
print(f"Mean: {df['price'].mean()}")
print(f"Standard deviation: {df['price'].std():.4f}")

0    199.0
1     89.0
2     99.0
3    129.0
4     79.0
Name: price, dtype: float64

Mean: 119.0
Standard deviation: 48.4768


In [44]:
(df["price"] - df["price"].mean()) / df["price"].std()

,price
0,1.650274
1,-0.618853
2,-0.412568
3,0.206284
4,-0.825137


In [45]:
def standardize(series: pd.Series):
  return (series - series.mean()) / series.std()

standardize(df["price"])

,price
0,1.650274
1,-0.618853
2,-0.412568
3,0.206284
4,-0.825137


In [46]:
scale(df["price"])

array([ 1.84506242, -0.69189841, -0.4612656 ,  0.2306328 , -0.92253121])

In [48]:
# Standard Scaler pozwala na zachowanie oryginalnych wartości i ich przywrócenie przez inverse_transform
scaler = StandardScaler()
scaler.fit(df[["price"]])
scaler.transform(df[["price"]])

array([[ 1.84506242],
       [-0.69189841],
       [-0.4612656 ],
       [ 0.2306328 ],
       [-0.92253121]])

In [50]:
scaler_price = StandardScaler()
scaler_price.fit_transform(df[["price"]])

array([[ 1.84506242],
       [-0.69189841],
       [-0.4612656 ],
       [ 0.2306328 ],
       [-0.92253121]])

In [53]:
scaler_many = StandardScaler()
df[["weight", "price"]] = scaler_many.fit_transform(df[["weight", "price"]])
df

,size,color,gender,price,weight,bought
0,XL,red,female,1.845062,1.366002,yes
1,L,green,male,-0.691898,0.623610,no
2,M,blue,male,-0.461266,-1.603567,yes
3,L,green,female,0.230633,-0.415740,no
4,M,red,female,-0.922531,0.029696,yes


In [59]:
dataframe = df_raw.copy()

le = LabelEncoder()
dataframe["bought"] = le.fit_transform(dataframe["bought"])

scaler = StandardScaler()
dataframe[["price", "weight"]] = scaler.fit_transform(dataframe[["price", "weight"]])

dataframe

,size,color,gender,price,weight,bought
0,XL,red,female,1.845062,1.366002,1
1,L,green,male,-0.691898,0.623610,0
2,M,blue,male,-0.461266,-1.603567,1
3,L,green,female,0.230633,-0.415740,0
4,M,red,female,-0.922531,0.029696,1


In [62]:
dataframe = pd.get_dummies(dataframe, drop_first=True)
dataframe

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,1,False,True,False,True,False
1,-0.691898,0.623610,0,False,False,True,False,True
2,-0.461266,-1.603567,1,True,False,False,False,True
3,0.230633,-0.415740,0,False,False,True,False,False
4,-0.922531,0.029696,1,True,False,False,True,False
